**Algoritmo Evolution Strategies em Python**

**Autor**: Iran Freitas Ribeiro

**Disciplina**: Computação Natural

**Professor**: Renato A. Krohling

Implementação baseada nos artigos: [Evolution strategies – A comprehensive introduction](https://link.springer.com/article/10.1023/A:1015059928466) e [Artigo](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.520.6737&rep=rep1&type=pdf)

In [1]:
from numpy import argsort
import numpy as np
import functions as f
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import rand_score
from sklearn.model_selection import train_test_split

In [13]:
def es_comma(data, n_clusters, bounds, n_iter, a, mu, lam, p, lograte=-1):
    """
    Executa o algoritmo de ES do tipo (mu, lambda)

    Params:
        - data: Dados originais
        - n_clusters: Número de clusters
        - bounds: limites dos espaço de busca
        - n_iter: número de gerações
        - a: força da mutação (step size)
        - mu: tamanho da população de pais
        - lam: tamanho da população de descendentes
        - p: numero de pais escolhidos para o processo de recombinação
        - lograte: taxa de exibição dos melhores resultados
    return:
        - best: melhor individuo
        - best_eval: score do melhor individuo
        - hist_scores: histórico de scores

    """
    # inicialização do melhor centroid e melhor score
    best, best_eval = None, 1e+10
    # inicia a população
    population = list(f.inicia_populacao(mu, bounds, n_clusters, data))
    hist_scores = []
    for epoch in range(n_iter):
        # score de cada individuo da população
        children = list()
        scores_children = list()    
        for l in range(lam):
            # selecao dos pais para o casamento
            pais_marriage, ids_ = f.marriage(population, p)
            # criação de um filho através da recombinação
            filho = f.recombination(pais_marriage)
            # mutação 
            filho = f.mutation(filho,a)
            # atualiza com o kmeans a cada 5 iterações            
            if (epoch%5==0):
                ## Inicia os centroids do kmeans usando o filho
                kmeans = KMeans(n_clusters=n_clusters, init=filho, n_init=1, max_iter=5)
                filho = kmeans.fit(data).cluster_centers_
            
            # avalia o filho otimizado com o kmeans
            scorefilho = f.eval_individual(filho, f.predict(data,filho), data)
            
            # atualiza o melhor score visto
            if scorefilho < best_eval:
                best_eval = scorefilho
                best = filho
            scores_children.append(scorefilho)
            children.append(filho)
            
        # ordena os scores do melhor para o maior
        ranks = argsort(argsort(scores_children))
        # seleciona os mu melhores filhos
        selected_children = [children[ranks[k]] for k,_ in enumerate(ranks[:mu])]
        if (lograte > 0 and epoch%lograte==0):
            print('%d, Best: %.5f' % (epoch, best_eval))
        population = selected_children
        hist_scores.append(best_eval)  
    return best, best_eval, hist_scores

In [9]:
def experiments(n_testes, exp_name = "", args=[], save=False):
    """
    Realiza os experimentos

    Params:
        - n_testes: número de execuções
        - exp_name: nome do experimento
        - args: lista de argumentos
    
    return:
        - best_exp: melhor individuo 
        - best_score_exp: score do melhor individuo
        - best_historico: historico de scores do melhor individuo

    """
    best_exp = None
    best_historico = []
    list_bests = []
    list_scores = []
    rand_index = []
    for i in range(n_testes):
        X_treino, X_teste, y_treino, y_teste = train_test_split(args[0], args[1], test_size=0.2)

        best, best_score, hist_scores = es_comma(X_treino,args[2], args[3], args[4], args[5], args[6], args[7], args[8])
        pred_labels = f.predict(X_teste, best)
        ri = rand_score(y_teste, pred_labels)
        rand_index.append(ri)
        
        list_bests.append(best)
        list_scores.append(best_score)
        best_historico.append(hist_scores)

        print ("Rand index {:.4f}".format(ri))
    print ("Media rand_index {:.4f}".format(np.mean(rand_index)))
    if save:
        np.save("results/ES/rand_index_{}.npy".format(exp_name), np.array(rand_index))        
        np.save('results/ES/best_historico_{}.npy'.format(exp_name), np.array(best_historico))
        np.save('results/ES/list_scores_{}.npy'.format(exp_name), np.array(list_scores))
        np.save('results/ES/best_exp_{}.npy'.format(exp_name),best_exp)
    return best_exp, best_historico

# Iris

In [5]:
iris_data, targets = f.load_iris()
labels_iris = iris_data[4].values
X_iris = iris_data[[0,1,2,3]].values

In [14]:
# definição dos argumentos
bounds = [0.1, 7.9]
n_clusters = 3
n_iter = 200
step_size = 0.015
mu = 40
lam = 110
p = 3
args_iris = [X_iris, labels_iris, n_clusters, bounds, n_iter, step_size, mu, lam, p]

In [15]:
_, _ = experiments(10, "es_kmeans_iris", args_iris)

Rand index 0.9632
Rand index 0.9310
Rand index 0.9218
Rand index 0.8621
Rand index 0.9264
Rand index 1.0000
Rand index 0.9172
Rand index 0.9218
Rand index 0.9080
Rand index 0.8345
Media rand_index 0.9186


# Wine dataset

In [16]:
wine_data, wine_targets = f.load_wine()
labels_wine = wine_data[0].values
wine_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [17]:
scaler_wine = MinMaxScaler()
wine_data_test= scaler_wine.fit_transform(wine_data[np.arange(1,14)])
X_wine = wine_data_test

In [18]:
# definição dos argumentos
bounds = [0.0, 1.0]
n_clusters = 3
n_iter = 200
step_size = 0.015
mu = 40
lam = 110
p=3
args_wine= [X_wine, labels_wine, n_clusters, bounds, n_iter, step_size, mu, lam, p]

In [19]:
_, _ = experiments(10, "es_kmeans_wine", args_wine)

Rand index 0.9635
Rand index 1.0000
Rand index 0.8873
Rand index 0.9270
Rand index 0.8921
Rand index 0.8873
Rand index 0.9619
Rand index 0.9603
Rand index 0.9286
Rand index 1.0000
Media rand_index 0.9408


# Breast Cancer Dataset

In [20]:
breast_cancer, targets_breast = f.load_breast_cancer()
colunas = breast_cancer.columns
# labels de cada linha da base de dados
labels = breast_cancer['diagnosis'].values
# seleciona apenas as features reais e pega os valores
data_breast_cancer = breast_cancer[colunas[1:]]
X_data_breast = data_breast_cancer.values

In [33]:
# transforma os dados para o intervalo [0,1]
scaler_breast = MinMaxScaler()
X_data_breast = scaler_breast.fit_transform(X_data_breast)

In [34]:
# definição dos argumentos
bounds = [0.0, 1.0]
n_clusters = len(targets_breast)
n_iter = 2000
step_size = 0.0015
mu = 40
lam = 110
p=3
args_breast= [X_data_breast, n_clusters, bounds, n_iter, step_size, mu, lam, p]

In [35]:
_, _, _ = experiments(10, "es_kmeans_breast", args_breast)

0 - best_score 318.12459827186456
1 - best_score 317.8322128258912
2 - best_score 317.8301896108884
3 - best_score 317.7991535749718
4 - best_score 317.76669729630123
5 - best_score 317.76669729630123
6 - best_score 317.76669729630123
7 - best_score 317.76669729630123
8 - best_score 317.76669729630123
9 - best_score 317.76669729630123
